In [8]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
import gc


from read_train import *
info = init_reading()

In [9]:

def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

def classic_sta_lta(x, length_sta, length_lta):
    
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Pad zeros
    sta[:length_lta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return sta / lta


def feature_extraction(raw_data):
    rows = 150000
    segments = int(np.floor(raw_data.shape[0] / rows))

    X_tr = pd.DataFrame(index=range(segments), dtype=np.float64)

    y_tr = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

    total_mean = raw_data['acoustic_data'].mean()
    total_std = raw_data['acoustic_data'].std()
    total_max = raw_data['acoustic_data'].max()
    total_min = raw_data['acoustic_data'].min()
    total_sum = raw_data['acoustic_data'].sum()
    total_abs_sum = np.abs(raw_data['acoustic_data']).sum()

    def calc_change_rate(x):
        change = (np.diff(x) / x[:-1]).values
        change = change[np.nonzero(change)[0]]
        change = change[~np.isnan(change)]
        change = change[change != -np.inf]
        change = change[change != np.inf]
        return np.mean(change)

    for segment in tqdm_notebook(range(segments)):
        seg = raw_data.iloc[segment*rows:segment*rows+rows]
        x = pd.Series(seg['acoustic_data'].values)
        y = seg['time_to_failure'].values[-1]

        y_tr.loc[segment, 'time_to_failure'] = y
        X_tr.loc[segment, 'mean'] = x.mean()
        X_tr.loc[segment, 'std'] = x.std()
        X_tr.loc[segment, 'max'] = x.max()
        X_tr.loc[segment, 'min'] = x.min()

        X_tr.loc[segment, 'mean_change_abs'] = np.mean(np.diff(x))
        X_tr.loc[segment, 'mean_change_rate'] = calc_change_rate(x)
        X_tr.loc[segment, 'abs_max'] = np.abs(x).max()
        X_tr.loc[segment, 'abs_min'] = np.abs(x).min()

        X_tr.loc[segment, 'std_first_50000'] = x[:50000].std()
        X_tr.loc[segment, 'std_last_50000'] = x[-50000:].std()
        X_tr.loc[segment, 'std_first_10000'] = x[:10000].std()
        X_tr.loc[segment, 'std_last_10000'] = x[-10000:].std()

        X_tr.loc[segment, 'avg_first_50000'] = x[:50000].mean()
        X_tr.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
        X_tr.loc[segment, 'avg_first_10000'] = x[:10000].mean()
        X_tr.loc[segment, 'avg_last_10000'] = x[-10000:].mean()

        X_tr.loc[segment, 'min_first_50000'] = x[:50000].min()
        X_tr.loc[segment, 'min_last_50000'] = x[-50000:].min()
        X_tr.loc[segment, 'min_first_10000'] = x[:10000].min()
        X_tr.loc[segment, 'min_last_10000'] = x[-10000:].min()

        X_tr.loc[segment, 'max_first_50000'] = x[:50000].max()
        X_tr.loc[segment, 'max_last_50000'] = x[-50000:].max()
        X_tr.loc[segment, 'max_first_10000'] = x[:10000].max()
        X_tr.loc[segment, 'max_last_10000'] = x[-10000:].max()

        X_tr.loc[segment, 'max_to_min'] = x.max() / np.abs(x.min())
        X_tr.loc[segment, 'max_to_min_diff'] = x.max() - np.abs(x.min())
        X_tr.loc[segment, 'count_big'] = len(x[np.abs(x) > 500])
        X_tr.loc[segment, 'sum'] = x.sum()

        X_tr.loc[segment, 'mean_change_rate_first_50000'] = calc_change_rate(x[:50000])
        X_tr.loc[segment, 'mean_change_rate_last_50000'] = calc_change_rate(x[-50000:])
        X_tr.loc[segment, 'mean_change_rate_first_10000'] = calc_change_rate(x[:10000])
        X_tr.loc[segment, 'mean_change_rate_last_10000'] = calc_change_rate(x[-10000:])

        X_tr.loc[segment, 'q95'] = np.quantile(x, 0.95)
        X_tr.loc[segment, 'q99'] = np.quantile(x, 0.99)
        X_tr.loc[segment, 'q05'] = np.quantile(x, 0.05)
        X_tr.loc[segment, 'q01'] = np.quantile(x, 0.01)

        X_tr.loc[segment, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
        X_tr.loc[segment, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
        X_tr.loc[segment, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
        X_tr.loc[segment, 'abs_q01'] = np.quantile(np.abs(x), 0.01)

        X_tr.loc[segment, 'trend'] = add_trend_feature(x)
        X_tr.loc[segment, 'abs_trend'] = add_trend_feature(x, abs_values=True)
        X_tr.loc[segment, 'abs_mean'] = np.abs(x).mean()
        X_tr.loc[segment, 'abs_std'] = np.abs(x).std()

        X_tr.loc[segment, 'mad'] = x.mad()
        X_tr.loc[segment, 'kurt'] = x.kurtosis()
        X_tr.loc[segment, 'skew'] = x.skew()
        X_tr.loc[segment, 'med'] = x.median()

        X_tr.loc[segment, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
        X_tr.loc[segment, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
        X_tr.loc[segment, 'classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
        X_tr.loc[segment, 'classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
        X_tr.loc[segment, 'classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
        X_tr.loc[segment, 'classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
        X_tr.loc[segment, 'classic_sta_lta5_mean'] = classic_sta_lta(x, 50, 1000).mean()
        X_tr.loc[segment, 'classic_sta_lta6_mean'] = classic_sta_lta(x, 100, 5000).mean()
        X_tr.loc[segment, 'classic_sta_lta7_mean'] = classic_sta_lta(x, 333, 666).mean()
        X_tr.loc[segment, 'classic_sta_lta8_mean'] = classic_sta_lta(x, 4000, 10000).mean()
        X_tr.loc[segment, 'Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
        ewma = pd.Series.ewm
        X_tr.loc[segment, 'exp_Moving_average_300_mean'] = (ewma(x, span=300).mean()).mean(skipna=True)
        X_tr.loc[segment, 'exp_Moving_average_3000_mean'] = ewma(x, span=3000).mean().mean(skipna=True)
        X_tr.loc[segment, 'exp_Moving_average_30000_mean'] = ewma(x, span=30000).mean().mean(skipna=True)
        no_of_std = 3
        X_tr.loc[segment, 'MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
        X_tr.loc[segment,'MA_700MA_BB_high_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] + no_of_std * X_tr.loc[segment, 'MA_700MA_std_mean']).mean()
        X_tr.loc[segment,'MA_700MA_BB_low_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] - no_of_std * X_tr.loc[segment, 'MA_700MA_std_mean']).mean()
        X_tr.loc[segment, 'MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
        X_tr.loc[segment,'MA_400MA_BB_high_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] + no_of_std * X_tr.loc[segment, 'MA_400MA_std_mean']).mean()
        X_tr.loc[segment,'MA_400MA_BB_low_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] - no_of_std * X_tr.loc[segment, 'MA_400MA_std_mean']).mean()
        X_tr.loc[segment, 'MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()
        X_tr.drop('Moving_average_700_mean', axis=1, inplace=True)

        X_tr.loc[segment, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
        X_tr.loc[segment, 'q999'] = np.quantile(x,0.999)
        X_tr.loc[segment, 'q001'] = np.quantile(x,0.001)
        X_tr.loc[segment, 'ave10'] = stats.trim_mean(x, 0.1)

        for windows in [10, 100, 1000]:
            x_roll_std = x.rolling(windows).std().dropna().values
            x_roll_mean = x.rolling(windows).mean().dropna().values

            X_tr.loc[segment, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
            X_tr.loc[segment, 'std_roll_std_' + str(windows)] = x_roll_std.std()
            X_tr.loc[segment, 'max_roll_std_' + str(windows)] = x_roll_std.max()
            X_tr.loc[segment, 'min_roll_std_' + str(windows)] = x_roll_std.min()
            X_tr.loc[segment, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
            X_tr.loc[segment, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
            X_tr.loc[segment, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
            X_tr.loc[segment, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
            X_tr.loc[segment, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
            X_tr.loc[segment, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
            X_tr.loc[segment, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()

            X_tr.loc[segment, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
            X_tr.loc[segment, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
            X_tr.loc[segment, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
            X_tr.loc[segment, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
            X_tr.loc[segment, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
            X_tr.loc[segment, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
            X_tr.loc[segment, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
            X_tr.loc[segment, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
            X_tr.loc[segment, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
            X_tr.loc[segment, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
            X_tr.loc[segment, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()
    return X_tr, y_tr

In [11]:
final_X = pd.DataFrame()
final_y = pd.DataFrame()
for i in range(17):
    train = read_object_info(info, i)
    X_tr, y_tr = feature_extraction(train)
    final_X = final_X.append(X_tr)
    final_y = final_y.append(y_tr)


In [12]:
final_X.to_csv("dataset/features.csv")
final_y.to_csv("dataset/targets.csv")

In [17]:
X = pd.read_csv("dataset/features.csv").drop("Unnamed: 0", axis=1)
X

,mean,std,max,min,mean_change_abs,mean_change_rate,abs_max,abs_min,std_first_50000,std_last_50000,...,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
0,4.884113,5.101106,104.0,-98.0,-0.000080,0.036138,104.0,0.0,6.488552,3.664663,...,0.295715,5.629,3.896,4.072,4.379,5.338,5.484,-1.704698e-06,74222.343443,5.629
1,4.725767,6.588824,181.0,-154.0,0.000000,0.022629,181.0,0.0,7.305233,5.493071,...,0.231587,5.667,3.412,4.233,4.345,5.066,5.223,-2.449664e-06,74364.426478,5.667
2,4.906393,6.967397,140.0,-106.0,-0.000013,0.023744,140.0,0.0,6.104836,8.603696,...,0.267012,5.957,4.055,4.239,4.446,5.344,5.486,1.140940e-06,74594.984584,5.957
3,4.902240,6.922305,197.0,-199.0,0.000000,0.042462,199.0,0.0,6.238109,5.652442,...,0.266700,5.858,3.722,4.304,4.433,5.317,5.453,-2.550336e-06,74333.143623,5.858
4,4.908720,7.301110,145.0,-126.0,-0.000007,0.038659,145.0,0.0,5.323830,7.694506,...,0.228005,6.078,3.918,4.434,4.543,5.306,5.491,1.550336e-06,74428.435969,6.078
5,4.913513,5.434111,142.0,-144.0,-0.000073,0.033474,144.0,0.0,4.442140,5.145255,...,0.293941,5.838,4.042,4.270,4.422,5.402,5.620,-3.697987e-06,74539.909434,5.838
6,4.855660,5.687823,120.0,-78.0,0.000033,0.028702,120.0,0.0,5.944368,5.386872,...,0.219472,5.546,3.958,4.285,4.504,5.214,5.370,-1.140940e-07,74501.904751,5.546
7,4.505427,5.854512,139.0,-134.0,0.000000,0.015607,139.0,0.0,6.069746,7.078177,...,0.218514,5.180,3.489,4.040,4.148,4.863,4.953,-1.395973e-06,74515.926219,5.180
8,4.717833,7.789643,168.0,-156.0,0.000013,-0.000942,168.0,0.0,9.288044,5.993102,...,0.240917,5.717,3.933,4.158,4.315,5.104,5.337,2.221477e-06,74464.251035,5.717
9,4.730960,6.890459,152.0,-126.0,-0.000007,0.014856,152.0,0.0,9.012695,4.944025,...,0.258462,5.423,3.832,4.190,4.301,5.128,5.261,5.127517e-06,74344.216475,5.423
